In [ ]:
import numpy as np
import math
import random
import timeit
from scipy.special import expit

#We will update the code depending on the data file in question
#Using the data file in question

#Line to import a .dat file
data = np.loadtxt('.dat file')

#Line to import a .csv file
data = np.genfromtxt('.csv file',delimiter=',')

for k in range(len(data)):
    #Where x represents the last column of the array
    if data[k][x]==2:
        data[k][x]=1 
    else:
        data[k][x]=-1

#Where x represents the first m rows, followed by the remaining rows in the data set, respectively
data_x = data[:x]
data_t = data[x:]
dtype_ = np.dtype(dtype="float64")

input_data=[]

#Where i and j represent the first n-1 columns, followed by the last column of the array, respectively
for r in range(len(data_x)):
    input_data.append([data_x[r][:i],data_x[r][j:]])

testing_data=[]

for s in range(len(data_t)):
    testing_data.append([data_t[s][:i],data_t[s][j:]])

#Setting up the data type for numpy arrays
dtype="float64"
dtype_ = np.dtype(dtype)
x_i = list()
y_i = list()
for i, j in input_data:
        x_i.append(i)
        y_i.append(j)
x_i, y_i = np.array(x_i, dtype=dtype_), np.array(y_i, dtype=dtype_)

#Initializing the values of the Omega and Beta
start_data_w= np.random.randint(np.size(x_i[0]), size=np.size(x_i[0]))
start_data_b = np.random.randint(np.size(y_i))
Alpha = 0.5
Beta = 0.5

#Gradient function
def gradient(training,W,B):
    L = 0.0001
    m = np.size(W)
    G0 = 2*L*W
    
    for x,y in input_data:
        #New array with the specified precision
        z = np.sum(-y*(np.sum(W*x[:, None]) + B))
        z = expit(z)
        sumgrad = np.subtract(np.sum(np.sum(-y*x)*np.exp(z)),np.sum(np.sum(y*np.exp(z))))/np.add(1,np.exp(np.sum(-y*np.exp(z))))
    return(np.add(G0,(1/m)*np.sum(sumgrad)))

def function_x(training_f, W, B):
    L = 0.0001
    m = np.size(W)
    
    for x,y in input_data:
        #New array with the specified precision
        z = np.sum(-y*(np.sum(W*x[:, None]) + B))
        z = expit(z)
        sumfun = np.log(1+np.exp(z))
    return(L*(np.linalg.norm(W))**2+(1/m)*np.sum(sumfun))            

def gradient_descent(
    gradient, training_d, start_w, start_b, stepsize, n_iter, tolerance,
    dtype="float64"
):
    start = timeit.default_timer()
    
    #Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    #Setting up the data type for numpy arrays
    dtype_ = np.dtype(dtype)

    #Converting x and y to numpy arrays
    x = list()
    y = list()
    for i, j in input_data:
        x.append(i)
        y.append(j)
        
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    if x.shape[0] != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")

    #Initializing the values of the variables
    vector = np.array(start_w, dtype=dtype_)
    beta = np.array(start_b, dtype=dtype_)

    #Setting up and checking the stepsize
    stepsize = np.array(stepsize, dtype=dtype_)
    if np.any(stepsize <= 0):
        raise ValueError("'stepsize' must be greater than zero")

    #Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    #Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")
    
    #Performing the gradient descent loop
    for _ in range(n_iter):
        #Recalculating the difference
        diff = -stepsize* np.array(gradient(training_d, vector, beta), dtype_)

        #Checking if the absolute difference is small enough and recalculating the step size
        if np.all(np.subtract(np.array(function_x(training_d, start_w, start_b), dtype_),np.array(function_x(diff, vector, beta), dtype_)))<Alpha*stepsize*np.linalg.norm(np.array(gradient(diff, vector, beta), dtype_))**2:
            #Updating the values of the variables
            stepsize = Beta*stepsize
        else:
            print("break",start,stop)
            break
            
        if np.all(np.abs(diff) > tolerance):
            vector, beta = (val + delta for val, delta in zip((vector, beta), diff))
        else:
            print("break",start,stop)
            break
            
        print(vector,beta)    
    count_accuracy = 0
    
    for h,l in testing_data:
        if np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))>=0 and np.array(l)==1:
            count_accuracy += 1

        elif np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))<0 and np.array(l)==-1:
            count_accuracy += 1
            
        check_accuracy = np.divide(count_accuracy,len(testing_data))*100
        
    stop = timeit.default_timer()
    
    return(vector if vector.shape else vector.item(), 
           print("The local minimum occurs at: Omega={}".format(vector), "Beta={}".format(beta)),
          print("Reported accuracy:{}%".format(check_accuracy)), print('Run time is: ', stop-start))

print(gradient_descent(gradient, input_data, start_data_w, start_data_b, 1, 500, 1e-04))